Settings

In [1]:
import zarr
import numpy as np
from dataset_util import get_value_target
from training.train_cnn import fill_default_config
from preprocessing.game_phase_detector import *


read_path = "/PommerLearn/data/samples/raw/1M_sample_0.zr"
write_path = "/PommerLearn/data/samples/steps/1M_sample_0"


/opt/miniconda3/envs/pommer/lib/python3.7/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Import error GL! You will not be able to render --> Library "GL" not found.


In [2]:
z = zarr.open(read_path)

chunks = z['act'].chunks

Generate Values

In [3]:
train_config = fill_default_config({
    "device":1,
})

val_target = get_value_target(
    z,
    discount_factor=train_config["discount_factor"], 
    mcts_val_weight=train_config["mcts_val_weight"]
    )
assert val_target.shape == z['val'].shape, "Shapes do not line up"

z['val']= val_target


Add phase Vector

In [4]:


get_phase_vector(z, phase_definition="steps")

100%|██████████| 1000000/1000000 [00:09<00:00, 102454.32it/s]


Split into three seperate zarrs

In [5]:
act = z['act'][:]
obs = z['obs'][:]
phase = z['phase'][:]
pol = z['pol'][:]
q = z['q'][:]
val = z['val'][:]

for i in range(2):
    mask = phase == i

    act_temp = act[mask]
    obs_temp = obs[mask]
    phase_temp = phase[mask]
    pol_temp = pol[mask]
    q_temp = q[mask]
    val_temp = val[mask]

    new_zarr = zarr.open(write_path+'_phase_'+str(i)+'.zr', mode='w')
    new_zarr.create_dataset("act", data=act_temp, chunks=chunks)
    new_zarr.create_dataset("obs", data=obs_temp, chunks=chunks)
    new_zarr.create_dataset("phase", data=phase_temp, chunks=chunks)
    new_zarr.create_dataset("pol", data=pol_temp, chunks=chunks)
    new_zarr.create_dataset("q", data=q_temp, chunks=chunks)
    new_zarr.create_dataset("val", data=val_temp, chunks=chunks)

    new_zarr.attrs['Steps'] = act.shape[0]
    new_zarr.attrs['AgentSteps'] = z.attrs['AgentSteps']
    new_zarr.attrs['EpisodeSteps'] = z.attrs['EpisodeSteps']
    new_zarr.attrs['AgentIds'] = z.attrs['AgentIds']
    new_zarr.attrs['AgentEpisode'] = z.attrs['AgentEpisode']
    new_zarr.attrs['EpisodeDead'] = z.attrs['EpisodeDead']
    new_zarr.attrs['EpisodeWinner'] = z.attrs['EpisodeWinner']
    new_zarr.attrs['EpisodeGameMode'] = z.attrs['EpisodeGameMode']
    new_zarr.attrs['EpisodeDraw'] = z.attrs['EpisodeDraw']
    new_zarr.attrs['EpisodeWinningTeam'] = z.attrs['EpisodeWinningTeam']
    new_zarr.attrs['EpisodeDone'] = z.attrs['EpisodeDone']
    new_zarr.attrs['EpisodeActions'] = z.attrs['EpisodeActions']


    print("Phase {} has {} items".format(i, act.shape[0])  )


Phase 0 has 252543 items
Phase 1 has 747457 items
